# **Pandas Data Transformations**

In [352]:
# group by and aggregations:: Data transformations is bridge between raw data and actionable insights.
import pandas as pd
penguins_body_mass=pd.read_csv("..\Datasets\penguins.csv")

# Aggregations in pandas

Aggregation operations summarize data within groups, reducing multiple values to a single value per group. They are a cornerstone of the split-apply-combine strategy in pandas' groupby:

- Split: Break the DataFrame into groups (e.g., by `body_mass_g`).
- Apply: Compute a summary (e.g., sum, mean, count) for each group.
- Combine: Collect results into a new DataFrame or Series.

Basic aggregation functions include:
- Numeric: `mean()`, `sum()`, `min()`, `max()`, `std()`, `median()`.
- General: `count()` (counts non-NaN values), `nunique()` (counts unique values), `size()` (counts total rows including NaNs).
- Custom: Use `.agg()` with functions or lambdas, e.g., `.agg(lambda x: max(x) - min(x))` for range, or `.agg(list)` to collect values.

Aggregations combine multiple values into a single result for each group, enabling efficient high-level insights into the data.


In [353]:
# Group data by 'species' and calculate the mean body mass for each group.
# This groups the penguins by species, then finds the average weight in grams.

penguins_body_mass.groupby(["species"])["body_mass_g"].mean()

species
Adelie       3700.662252
Chinstrap    3733.088235
Gentoo       5076.016260
Name: body_mass_g, dtype: float64

The code :
penguins_body_mass.groupby(["body_mass_g"])["species"].mean()

did not work because the `.mean()` function in pandas requires numeric data and fails on strings.

- `.mean()` → works only on numeric data, so it raises an error or returns nothing when applied to non-numeric columns like strings (e.g., `"species"`).
- `.sum()` → with strings, it concatenates the values instead of summing numerically.
- `.min()` / `.max()` → applied to strings, these return the lexicographically smallest or largest value respectively.

Thus, `.mean()` is stricter and numeric-only, whereas `.sum()`, `.min()`, and `.max()` can operate on string data but with behavior that reflects string operations rather than arithmetic.

In [354]:
# Get unique species values (3 unique species in dataset)
penguins_body_mass["species"].unique()
# three unique values. 

array(['Adelie', 'Gentoo', 'Chinstrap'], dtype=object)

In [355]:
# Using agg() to compute multiple aggregation metrics for each species
penguins_body_mass.groupby("species")["flipper_length_mm"].agg(["sum","mean"])

,sum,mean
species,,
Adelie,28683.0,189.953642
Chinstrap,13316.0,195.823529
Gentoo,26714.0,217.186992


In [356]:
result = penguins_body_mass.groupby("species").agg({
    "flipper_length_mm": ["sum", "mean"],   # multiple aggregations for flipper length
    "body_mass_g": ["mean", "max", "min"]   # multiple aggregations for body mass
})
result

flipper_length_mm              body_mass_g                
                        sum        mean         mean     max     min
species                                                             
Adelie              28683.0  189.953642  3700.662252  4775.0  2850.0
Chinstrap           13316.0  195.823529  3733.088235  4800.0  2700.0
Gentoo              26714.0  217.186992  5076.016260  6300.0  3950.0

In [357]:
# Count non-null body_mass_g values for each species
penguins_body_mass.groupby("species")["body_mass_g"].count()

species
Adelie       151
Chinstrap     68
Gentoo       123
Name: body_mass_g, dtype: int64

In [358]:
# Advanced aggregations: applying multiple aggregation functions to summarize data.

aggregations= penguins_body_mass.groupby("island")["year"].agg(["max","min"])
aggregations

,max,min
island,,
Biscoe,2009,2007
Dream,2009,2007
Torgersen,2009,2007


In [359]:
# To make results clearer, we can rename aggregation columns using a dictionary.
renamed_columns={
    "maximum year":"max",
    "average_year":"mean"
}
aggregations= penguins_body_mass.groupby("species")["year"].agg(**renamed_columns)
aggregations

,maximum year,average_year
species,,
Adelie,2009,2008.013158
Chinstrap,2009,2007.970588
Gentoo,2009,2008.080645


In [360]:
# Applying different aggregation functions to multiple columns.
aggregations= penguins_body_mass.groupby("island")["year"].agg(max_year="max",min_year="min")
aggregations

,max_year,min_year
island,,
Biscoe,2009,2007
Dream,2009,2007
Torgersen,2009,2007


In [361]:
# Applying different aggregation functions to multiple columns.
penguins_body_mass.groupby("island").agg({"flipper_length_mm":"min", "bill_length_mm":"min"})

,flipper_length_mm,bill_length_mm
island,,
Biscoe,172.0,34.5
Dream,178.0,32.1
Torgersen,176.0,33.5


In [362]:
# Combine multiple aggregations into one grouped summary.
penguins_body_mass.groupby("island").agg(
    average_body_mass=("body_mass_g", "mean"),
    unique_species_count=("species", "nunique"),
    std_body_mass=("body_mass_g", "std") #standard deviation 
)

,average_body_mass,unique_species_count,std_body_mass
island,,,
Biscoe,4716.017964,2,782.855743
Dream,3712.903226,2,416.644112
Torgersen,3706.372549,1,445.107940


- **Aggregations** such as `sum()`, `mean()`, `min()`, `max()`, `count()`, and `nunique()` summarize data within groups.
- **Multiple aggregations** can be applied simultaneously using `.agg()` with lists, dictionaries, or named aggregation syntax.
- **Named aggregations** allow assigning descriptive names to output columns for clarity.
- Proper data types (numeric for math functions) are crucial to avoid errors during aggregation.
- For categorical data, functions like `nunique` or `count` provide meaningful insights rather than numeric aggregates like `sum`.

## Best Practices
- Use **named aggregations** for clean, readable output:
- When aggregating strings, avoid numeric functions; use counts or unique counts instead.
- Rename columns either during aggregation (named aggregation) or afterwards using `.rename()`.
- Examine and convert data types before aggregation to prevent errors like `TypeError: agg function failed`.

## Summary Table of Useful Aggregations

| Function      | Description                             | Use Case                |
|---------------|-------------------------------------|-------------------------|
| `sum()`       | Sum of values                       | Numeric totals          |
| `mean()`      | Arithmetic average                  | Central tendency        |
| `min()`/`max()`| Minimum/Maximum values              | Range insights          |
| `count()`     | Non-null count                     | Number of entries       |
| `nunique()`   | Count of unique values             | Category diversity      |
| `std()`       | Standard deviation                 | Data variability        |

Mastering `groupby` and `agg` empowers efficient exploratory data analysis and reporting by summarizing complex datasets into comprehensible insights. Always choose aggregation functions considering data types and the analysis context for meaningful results.

In [363]:
# using custom Aggregations functions: custom functions helps us to do advanced action tailored to our specific names.
import pandas as pd
penguins_data=pd.read_csv("../Datasets/penguins.csv")

## why custom Aggregations functions :?
- When built-in functions doesnot meet our requirements
- To implement specific business rules for analysis. 
-  Advanced metrices such as ranges, percentages are required.

### Key Methods

- **Basic Syntax:**  
  ```
  df.groupby('column')['column_to_aggregate'].agg(your_function)
  ```

- **Lambda:**  
  ```
  agg(lambda x: max(x) - min(x))
  ```

- **Named Function:**  
  ```
  def my_func(x):
      return ...
  agg(my_func)
  ```

- **Multiple Functions:**  
  ```
  agg([my_func, 'mean'])
  # or
  agg({'column': [(name, my_func)]})  # for named outputs
  ```
```


In [364]:
def calculate_range(series):
    return series.max()-series.min()
custom_aggregations = penguins_data.groupby("species")["body_mass_g"].agg(
    unique_species= "nunique",
    mass_min="min",
    mass_max="max",
    mass_range=calculate_range 
)
custom_aggregations

# The column used for aggregation is determined by what you select after the groupby (here body_mass_g).

# The aggregation functions (like min, max) are applied to this selected column group-wise.

# The output column names inside .agg() come from the keyword argument names you provide.

,unique_species,mass_min,mass_max,mass_range
species,,,,
Adelie,55,2850.0,4775.0,1925.0
Chinstrap,34,2700.0,4800.0,2100.0
Gentoo,47,3950.0,6300.0,2350.0


In [365]:
custom_agg = penguins_data.groupby('island').agg(
    unique_species=('species', 'nunique'),
    mass_range=('body_mass_g', lambda s: s.max() - s.min())
)
custom_agg

# Groups penguins by island.
# Counts unique species in each island group.
# Calculates range (max - min) of body_mass_g per island.

,unique_species,mass_range
island,,
Biscoe,2,3450.0
Dream,2,2100.0
Torgersen,1,1800.0


In [366]:
# Custom function: count values above 4000
def count_heavy(x):
    return (x > 4000).sum()

result = penguins_body_mass.groupby('species')['body_mass_g'].agg(count_heavy)
print(result)

species
Adelie        35
Chinstrap     15
Gentoo       122
Name: body_mass_g, dtype: int64


In [367]:
# custom complex aggregarions
penguins_adelie=penguins_body_mass.query("species=='Adelie'")
len(penguins_adelie["island"])

152

In [368]:
sum(penguins_adelie["island"]=="Biscoe")
# .sum() on this mask counts the number of True values (matches), while len() just returns total length unrelated to matching count.

44

To calculate the percentage of each species within a pandas DataFrame using custom complex aggregations, you can apply a groupby operation combined with a custom aggregation function. The general approach is:

1. Group the DataFrame by the categorical column (`species`).
2. Count the number of entries in each group.
3. Divide each group's count by the total count of all entries (to get the proportion).
4. Multiply by 100 to convert the proportion to a percentage.

Here is an example code snippet implementing this:

In [369]:
def percentage_of_total(x):
    return (len(x) / len(penguins_body_mass)) * 100
result = penguins_body_mass.groupby("species").agg(percentage=('species', percentage_of_total))
result

,percentage
species,
Adelie,44.186047
Chinstrap,19.767442
Gentoo,36.046512


In [370]:
result=penguins_body_mass.groupby("island").agg(percentage=("island",percentage_of_total))
result

,percentage
island,
Biscoe,48.837209
Dream,36.046512
Torgersen,15.116279


In [371]:
def percentage_of_total(x):
    return (len(x) / len(penguins_body_mass)) * 100

# Group by both 'island' and 'species',
# then apply the function to count rows per group and convert to percentage of total
result = penguins_body_mass.groupby(['island', 'species']).agg(percentage=('species', percentage_of_total))
result.reset_index()


,island,species,percentage
0,Biscoe,Adelie,12.790698
1,Biscoe,Gentoo,36.046512
2,Dream,Adelie,16.279070
3,Dream,Chinstrap,19.767442
4,Torgersen,Adelie,15.116279


# **How to use Apply function for Transformations**:

Transformations involve applying functions to modify or create new data in a DataFrame or Series, such as:

- Scaling numeric values (e.g., normalizing body_mass_g).
- Modifying strings (e.g., capitalizing species).
- Creating new columns based on row-wise calculations (e.g., body mass index).
- Applying group-based logic (e.g., subtracting the group mean).

Unlike aggregations (which reduce data, like `mean()` per group), transformations preserve the shape of the data (same number of rows or columns) or create new columns. Pandas provides several methods to apply functions for transformations:

- **`apply()`**: General-purpose, applies functions to rows, columns, or entire DataFrames.
- **`map()`**: Applies functions to each element in a Series (column).
- **`applymap()`**: Applies functions element-wise to an entire DataFrame (deprecated in favor of `map` for Series or `apply` with lambda in newer pandas versions).
- **`transform()`**: Applies group-wise transformations, aligning results with the original DataFrame’s index.
- **Vectorized Operations**: Use pandas/NumPy operations for faster, element-wise transformations without explicit loops.


# Using `apply()` for Transformations in Pandas
Use `apply()` to apply custom functions to rows or columns, preserving data shape. 

- **Key Methods**:
  - Row-wise: `df.apply(func, axis=1)`
  - Column-wise: `df.apply(func, axis=0)`
  - Multiple outputs: Return `pd.Series` in the function

In [372]:
# how to use apply.  
import pandas as pd
penguins=pd.read_csv("..\Datasets\penguins.csv")            

In [373]:
# Calculate BMI: Row wise
def calculate_bmi(row):
    return row["body_mass_g"]/ (row["flipper_length_mm"]/1000)**2
penguins["bmi"]=penguins.apply(calculate_bmi,axis=1)
penguins

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bmi
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,114465.370410
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,109839.287779
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,85470.085470
3,4,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007,NaN
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,92619.936106
...,...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,93351.070037
340,341,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009,83325.164200
341,342,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,101345.002550
342,343,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009,92970.521542


In [374]:
#Use pd.notnull() to check for non-null values before calculation.

def calculate_bmi(row):
    if pd.notnull(row["body_mass_g"]) and pd.notnull(row["flipper_length_mm"]):
        return row["body_mass_g"] / (row["flipper_length_mm"] / 1000) ** 2
    else:
        return None
penguins["bmi"]=penguins.apply(calculate_bmi,axis=1)
penguins

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,bmi
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,114465.370410
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,109839.287779
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,85470.085470
3,4,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007,NaN
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,92619.936106
...,...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,93351.070037
340,341,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009,83325.164200
341,342,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,101345.002550
342,343,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009,92970.521542


In [375]:
# Size category
def heavy(size):
    return size["body_mass_g"]>4000
penguins["weighty"]=penguins.apply(heavy,axis=1)
penguins.drop(["bill_length_mm","bill_depth_mm","flipper_length_mm"],axis=1) #just to make output clear. 

,rowid,species,island,body_mass_g,sex,year,bmi,weighty
0,1,Adelie,Torgersen,3750.0,male,2007,114465.370410,False
1,2,Adelie,Torgersen,3800.0,female,2007,109839.287779,False
2,3,Adelie,Torgersen,3250.0,female,2007,85470.085470,False
3,4,Adelie,Torgersen,NaN,NaN,2007,NaN,False
4,5,Adelie,Torgersen,3450.0,female,2007,92619.936106,False
...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,4000.0,male,2009,93351.070037,False
340,341,Chinstrap,Dream,3400.0,female,2009,83325.164200,False
341,342,Chinstrap,Dream,3775.0,male,2009,101345.002550,False
342,343,Chinstrap,Dream,4100.0,male,2009,92970.521542,True


In [376]:
# Conditional Size Category:

def size_category(row):
    mass=row["body_mass_g"]
    flipper=row["flipper_length_mm"]
    if mass>4000 and flipper>200:
        return "large"
    elif mass<3600:
        return "Medium"
    return "Small"
penguins['size_category'] = penguins.apply(size_category, axis=1)
penguins.drop(["bill_length_mm","bill_depth_mm","flipper_length_mm"],axis=1) #just to make output clear. 

,rowid,species,island,body_mass_g,sex,year,bmi,weighty,size_category
0,1,Adelie,Torgersen,3750.0,male,2007,114465.370410,False,Small
1,2,Adelie,Torgersen,3800.0,female,2007,109839.287779,False,Small
2,3,Adelie,Torgersen,3250.0,female,2007,85470.085470,False,Medium
3,4,Adelie,Torgersen,NaN,NaN,2007,NaN,False,Small
4,5,Adelie,Torgersen,3450.0,female,2007,92619.936106,False,Medium
...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,4000.0,male,2009,93351.070037,False,Small
340,341,Chinstrap,Dream,3400.0,female,2009,83325.164200,False,Medium
341,342,Chinstrap,Dream,3775.0,male,2009,101345.002550,False,Small
342,343,Chinstrap,Dream,4100.0,male,2009,92970.521542,True,large


In [377]:
# Multiple Outputs:

def mass_features(row):
    mass=row["body_mass_g"]
    max_mass=penguins["body_mass_g"].max()
    return pd.Series({
        'mass_normalized': (mass - penguins['body_mass_g'].min()) / (max_mass - penguins['body_mass_g'].min()),
        'is_heavy': 1 if mass > 4000 else 0
    })
penguins[['mass_normalized', 'is_heavy']] = penguins.apply(mass_features, axis=1)
penguins.drop(["bill_length_mm","bill_depth_mm","flipper_length_mm"],axis=1) #just to make output clear. 

,rowid,species,island,body_mass_g,sex,year,bmi,weighty,size_category,mass_normalized,is_heavy
0,1,Adelie,Torgersen,3750.0,male,2007,114465.370410,False,Small,0.291667,0.0
1,2,Adelie,Torgersen,3800.0,female,2007,109839.287779,False,Small,0.305556,0.0
2,3,Adelie,Torgersen,3250.0,female,2007,85470.085470,False,Medium,0.152778,0.0
3,4,Adelie,Torgersen,NaN,NaN,2007,NaN,False,Small,NaN,0.0
4,5,Adelie,Torgersen,3450.0,female,2007,92619.936106,False,Medium,0.208333,0.0
...,...,...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,4000.0,male,2009,93351.070037,False,Small,0.361111,0.0
340,341,Chinstrap,Dream,3400.0,female,2009,83325.164200,False,Medium,0.194444,0.0
341,342,Chinstrap,Dream,3775.0,male,2009,101345.002550,False,Small,0.298611,0.0
342,343,Chinstrap,Dream,4100.0,male,2009,92970.521542,True,large,0.388889,1.0


# **Adding and Updating column in Pandas**

In [378]:
import pandas as pd
penguins=pd.read_csv("..\Datasets\penguins.csv")
penguins.head(n=4)

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,4,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007


In [379]:
# fillna for filling all Non -values
mean_value=penguins["body_mass_g"].mean()
penguins["body_mass_g"]=penguins["body_mass_g"].fillna(mean_value)
penguins.head(n=5)

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.000000,male,2007
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.000000,female,2007
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.000000,female,2007
3,4,Adelie,Torgersen,NaN,NaN,NaN,4201.754386,NaN,2007
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.000000,female,2007


In [380]:
# adding new column: 
penguins["body_mass_kg"]=penguins["body_mass_g"]/1000
penguins

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,body_mass_kg
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.000000,male,2007,3.750000
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.000000,female,2007,3.800000
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.000000,female,2007,3.250000
3,4,Adelie,Torgersen,NaN,NaN,NaN,4201.754386,NaN,2007,4.201754
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.000000,female,2007,3.450000
...,...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,55.8,19.8,207.0,4000.000000,male,2009,4.000000
340,341,Chinstrap,Dream,43.5,18.1,202.0,3400.000000,female,2009,3.400000
341,342,Chinstrap,Dream,49.6,18.2,193.0,3775.000000,male,2009,3.775000
342,343,Chinstrap,Dream,50.8,19.0,210.0,4100.000000,male,2009,4.100000


In [381]:
# Category:
penguins["Gender_label"]="male"
penguins.loc[penguins["sex"]=="female","Gender_label"]="female"
penguins.drop("sex",axis=1) #dropping sex column  from the Dataframe. 


,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year,body_mass_kg,Gender_label
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.000000,2007,3.750000,male
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.000000,2007,3.800000,female
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.000000,2007,3.250000,female
3,4,Adelie,Torgersen,NaN,NaN,NaN,4201.754386,2007,4.201754,male
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.000000,2007,3.450000,female
...,...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,55.8,19.8,207.0,4000.000000,2009,4.000000,male
340,341,Chinstrap,Dream,43.5,18.1,202.0,3400.000000,2009,3.400000,female
341,342,Chinstrap,Dream,49.6,18.2,193.0,3775.000000,2009,3.775000,male
342,343,Chinstrap,Dream,50.8,19.0,210.0,4100.000000,2009,4.100000,male


In [382]:
#categorized by flipper length
mean_flipper=penguins["flipper_length_mm"].mean()
penguins["flipper_length_mm"]=penguins["flipper_length_mm"].fillna(mean_flipper)
penguins["size"]="medium"
penguins.loc[penguins["flipper_length_mm"]>205,"size"]="big"
penguins.loc[penguins["flipper_length_mm"]<185,"size"]="small"
penguins.head(n=10)

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,body_mass_kg,Gender_label,size
0,1,Adelie,Torgersen,39.1,18.7,181.000000,3750.000000,male,2007,3.750000,male,small
1,2,Adelie,Torgersen,39.5,17.4,186.000000,3800.000000,female,2007,3.800000,female,medium
2,3,Adelie,Torgersen,40.3,18.0,195.000000,3250.000000,female,2007,3.250000,female,medium
3,4,Adelie,Torgersen,NaN,NaN,200.915205,4201.754386,NaN,2007,4.201754,male,medium
4,5,Adelie,Torgersen,36.7,19.3,193.000000,3450.000000,female,2007,3.450000,female,medium
5,6,Adelie,Torgersen,39.3,20.6,190.000000,3650.000000,male,2007,3.650000,male,medium
6,7,Adelie,Torgersen,38.9,17.8,181.000000,3625.000000,female,2007,3.625000,female,small
7,8,Adelie,Torgersen,39.2,19.6,195.000000,4675.000000,male,2007,4.675000,male,medium
8,9,Adelie,Torgersen,34.1,18.1,193.000000,3475.000000,NaN,2007,3.475000,male,medium
9,10,Adelie,Torgersen,42.0,20.2,190.000000,4250.000000,NaN,2007,4.250000,male,medium


In [383]:
# on making functions for same problem:

# to convert in numeric, coercing errors to NaN. 
penguins["flipper_length_mm"] = pd.to_numeric(penguins["flipper_length_mm"], errors='coerce')
mean_flipper = penguins["flipper_length_mm"].mean()
penguins["flipper_length_mm"] = penguins["flipper_length_mm"].fillna(mean_flipper)
def get_size_category(size):
    if size["flipper_length_mm"]>205:
        return "Big"
    elif size["flipper_length_mm"]<185:
        return "small"
    else:
        return "medium"
penguins["size"]=penguins.apply(get_size_category,axis=1) # axis =1, for each and every row. 
penguins.head()

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,body_mass_kg,Gender_label,size
0,1,Adelie,Torgersen,39.1,18.7,181.000000,3750.000000,male,2007,3.750000,male,small
1,2,Adelie,Torgersen,39.5,17.4,186.000000,3800.000000,female,2007,3.800000,female,medium
2,3,Adelie,Torgersen,40.3,18.0,195.000000,3250.000000,female,2007,3.250000,female,medium
3,4,Adelie,Torgersen,NaN,NaN,200.915205,4201.754386,NaN,2007,4.201754,male,medium
4,5,Adelie,Torgersen,36.7,19.3,193.000000,3450.000000,female,2007,3.450000,female,medium


In [384]:
# Replace method:
penguins["Gender_label"]=penguins["Gender_label"].replace({"male":"Male", "female":"Female"})
penguins.head()

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,body_mass_kg,Gender_label,size
0,1,Adelie,Torgersen,39.1,18.7,181.000000,3750.000000,male,2007,3.750000,Male,small
1,2,Adelie,Torgersen,39.5,17.4,186.000000,3800.000000,female,2007,3.800000,Female,medium
2,3,Adelie,Torgersen,40.3,18.0,195.000000,3250.000000,female,2007,3.250000,Female,medium
3,4,Adelie,Torgersen,NaN,NaN,200.915205,4201.754386,NaN,2007,4.201754,Male,medium
4,5,Adelie,Torgersen,36.7,19.3,193.000000,3450.000000,female,2007,3.450000,Female,medium


# **Chaining Transformation**
Chaining in Pandas is like: combining multiple DataFrame operations into a single, readable line of code, where each step transforms the data and passes it to the next. Instead of writing clunky, separate lines that create temporary variables, chaining lets you streamline your workflow, making your code cleaner and more **Pythonic**.

In Pandas, `chaining` means applying methods like filtering, grouping, or adding columns sequentially using the dot (`.`) notation. This approach is powerful for data cleaning and analysis because it allows us to combine multiple operations into a single, readable line of code, improving code conciseness and clarity.

In [385]:
import pandas as pd
penguins=pd.read_csv("..\Datasets\penguins.csv")
penguins.head()

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,4,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


## NOTE :
# Why Use `reset_index()` and `.copy()` in pandas?

When working with pandas DataFrames, especially after filtering or slicing, understanding `reset_index()` and `.copy()` is important for clean and error-free data handling.



## 1. What does `reset_index()` do?

- When you filter a DataFrame (e.g., select rows where a condition is true), pandas **keeps the original row indices**.
- This means your DataFrame may have row indices like `[3, 5, 8]` instead of `[0, 1, 2]`.
- Using `reset_index()` resets the index to be sequential starting from zero, which is often easier to work with.


## 2. Why use `.copy()`?

- Filtering a DataFrame sometimes returns a **view** of the original data, not a separate copy.
- Modifying a view can trigger warnings ("SettingWithCopyWarning") or unintended side effects.
- Using `.copy()` creates a **new, independent DataFrame** safe to modify.

In [386]:
penguins = penguins.reset_index()
male=penguins[penguins["sex"]=="male"].copy()
male.head()

,index,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
5,5,6,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007
7,7,8,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male,2007
13,13,14,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male,2007
14,14,15,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male,2007


In [387]:
# Filter the DataFrame to include only male penguins and add a new column 'body_mass_kg' by converting 'body_mass_g' to kilograms
body_mass_kg=(
    penguins[penguins["sex"]=="male"]
    .assign(body_mass_kg=lambda df: round(df["body_mass_g"] / 1000))
    ) #rounding off values as well 
body_mass_kg

,index,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,body_mass_kg
0,0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,4.0
5,5,6,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,4.0
7,7,8,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male,2007,5.0
13,13,14,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male,2007,4.0
14,14,15,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male,2007,4.0
...,...,...,...,...,...,...,...,...,...,...,...
334,334,335,Chinstrap,Dream,50.2,18.8,202.0,3800.0,male,2009,4.0
336,336,337,Chinstrap,Dream,51.9,19.5,206.0,3950.0,male,2009,4.0
339,339,340,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,4.0
341,341,342,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,4.0


In [388]:
merged=(
    pd.merge(body_mass_kg,male, left_on='index',right_on='index')
)
merged

,index,rowid_x,species_x,island_x,bill_length_mm_x,bill_depth_mm_x,flipper_length_mm_x,body_mass_g_x,sex_x,year_x,body_mass_kg,rowid_y,species_y,island_y,bill_length_mm_y,bill_depth_mm_y,flipper_length_mm_y,body_mass_g_y,sex_y,year_y
0,0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,4.0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,5,6,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,4.0,6,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007
2,7,8,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male,2007,5.0,8,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male,2007
3,13,14,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male,2007,4.0,14,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male,2007
4,14,15,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male,2007,4.0,15,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,334,335,Chinstrap,Dream,50.2,18.8,202.0,3800.0,male,2009,4.0,335,Chinstrap,Dream,50.2,18.8,202.0,3800.0,male,2009
164,336,337,Chinstrap,Dream,51.9,19.5,206.0,3950.0,male,2009,4.0,337,Chinstrap,Dream,51.9,19.5,206.0,3950.0,male,2009
165,339,340,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,4.0,340,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
166,341,342,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,4.0,342,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009



---

## Summary

| Concept       | Why Use It?                                | Example Usage                        |
|---------------|-------------------------------------------|------------------------------------|
| `reset_index` | To reset row numbering to sequential order | `df.reset_index(drop=True, inplace=True)` |
| `.copy()`     | To create an independent DataFrame copy  | `df_copy = df_original.copy()`     |

Using these two methods helps prevent index confusion and warnings while making your data clean and safe to change.


# Advanced Chaining Techniques

Chaining means applying multiple methods in one continious stream eg: `df.query(...).assign(...).groupby(...).agg(...)` .
<br>
It makes code compact, readable and aligned with data Transformations with logic. 


In [389]:
import pandas as pd
penguins=pd.read_csv("..\Datasets\penguins.csv")
penguins.head()

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,4,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [390]:
#checking non=values
missing_bill_length=penguins["bill_length_mm"].isnull().sum()
missing_bill_length

np.int64(2)

In [ ]:
penguins.fillna({"bill_length_mm":0}) \
    .query("sex=='male'").assign(body_mass_kg=lambda df: round(df["body_mass_g"]/1000))
# changed the data by replacing NaNs with 0 before filtering, which results in a different dataset and output than filtering first or leaving NaNs untouched.

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year,body_mass_kg
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,4.0
5,6,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007,4.0
7,8,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male,2007,5.0
13,14,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male,2007,4.0
14,15,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male,2007,4.0
...,...,...,...,...,...,...,...,...,...,...
334,335,Chinstrap,Dream,50.2,18.8,202.0,3800.0,male,2009,4.0
336,337,Chinstrap,Dream,51.9,19.5,206.0,3950.0,male,2009,4.0
339,340,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009,4.0
341,342,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009,4.0


In [407]:
df1 = penguins[penguins["sex"] == "female"].copy()
df2 = penguins.fillna({"bill_length_mm": 0}).copy()

df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

result = (
    pd.merge(df1, df2, left_index=True, right_index=True, suffixes=('_df1', '_df2'))
      .fillna({"bill_length_mm_df2": 0})
      .query("sex_df1 == 'female'") 
      .assign(body_mass_kg=lambda df: round(df["body_mass_g_df1"] / 1000))
)
result

,level_0_df1,index_df1,rowid_df1,species_df1,island_df1,bill_length_mm_df1,bill_depth_mm_df1,flipper_length_mm_df1,body_mass_g_df1,sex_df1,...,rowid_df2,species_df2,island_df2,bill_length_mm_df2,bill_depth_mm_df2,flipper_length_mm_df2,body_mass_g_df2,sex_df2,year_df2,body_mass_kg
0,1,1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,...,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007,4.0
1,2,2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,...,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007,3.0
2,4,4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,...,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007,3.0
3,6,6,7,Adelie,Torgersen,38.9,17.8,181.0,3625.0,female,...,4,Adelie,Torgersen,0.0,NaN,NaN,NaN,NaN,2007,4.0
4,12,12,13,Adelie,Torgersen,41.1,17.6,182.0,3200.0,female,...,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,335,335,336,Chinstrap,Dream,45.6,19.4,194.0,3525.0,female,...,161,Gentoo,Biscoe,43.3,13.4,209.0,4400.0,female,2007,4.0
161,337,337,338,Chinstrap,Dream,46.8,16.5,189.0,3650.0,female,...,162,Gentoo,Biscoe,46.8,15.4,215.0,5150.0,male,2007,4.0
162,338,338,339,Chinstrap,Dream,45.7,17.0,195.0,3650.0,female,...,163,Gentoo,Biscoe,40.9,13.7,214.0,4650.0,female,2007,4.0
163,340,340,341,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,...,164,Gentoo,Biscoe,49.0,16.1,216.0,5550.0,male,2007,3.0


# Pandas Method Chaining:
---

## Core Methods

- `.query()` — Filter rows with readable SQL-like conditions  
  Example: `df.query('sex == "male"')`

- `.assign()` — Add or modify columns inline  
  Example: `df.assign(body_mass_kg=lambda x: x.body_mass_g / 1000)`

- `.groupby()` — Group data by category for aggregation  
  Example: `df.groupby('species').mean()`

- `.agg()` — Apply multiple summary stats on grouped data  
  Example: `.agg({'body_mass_kg': 'mean', 'flipper_length_mm': 'max'})`

- `.pipe()` — Insert custom functions into your chain cleanly  
  Example: `df.pipe(custom_func)`

---
## Tips

- Use `.query()` for clear filtering.  
- Use `.assign()` to avoid intermediate variables.  
- Chain `.groupby()` and `.agg()` for summaries.  
- Use `.pipe()` for custom transformations. 
---